In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import random_split
from torchvision import datasets
from torchvision import transforms
import numpy as np
import pickle
from model import ZFNet 
import pandas as pd
import matplotlib as plt
import seaborn as sns
from tqdm.notebook import tqdm # позволяет делать прогресс бары, такадум с арабского означает прогресс :)

In [3]:
def train(model, train_dl, test_dl, opt, loss_func, epochs):
    """ train model using using provided datasets, optimizer and loss function """
    train_loss = [0 for i in range(epochs)]
    test_loss = [0 for i in range(epochs)]
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt)
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss = loss_func(model(xb), yb)
            train_loss[epoch] = loss.item()
            loss.backward()
            opt.step()
            opt.zero_grad()
        with torch.no_grad():
            losses, nums = zip(*[(loss_func(model(xb),yb).item(),len(xb)) for xb, yb in test_dl])
            test_loss[epoch] = np.sum(np.multiply(losses, nums)) / np.sum(nums)
            correct = 0
            total = 0
            for data in test_dl:
                images, labels = data
                images, labels = images, labels
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss[epoch]}, Test Loss {test_loss[epoch]}, Accuracy: {100*correct/total}')
        scheduler.step(test_loss[epoch])
    return train_loss, test_loss

if __name__ == "__main__":
    # Define transforms
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    # Load training data
    dataset = datasets.ImageFolder('/home/julia/Documents/python/sod/train_resized', transform=transform) 
    test_data, train_data = random_split(dataset,(1547, 4641), generator=torch.Generator().manual_seed(42))
    train_dl = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
    test_dl = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=4)
    # Train Model
    epochs = 5
    model = ZFNet()
    loss_func = nn.CrossEntropyLoss()
    opt = optim.Adam(model.parameters(), lr=0.0001)
    train_loss, test_loss = train(model, train_dl, test_dl, opt, loss_func, epochs)
    # Save Model to pkl file
    save_path = '/home/julia/Documents/python/sod/ans/model.pkl'
    with open(save_path, 'wb') as f:
        pickle.dump(model, f)
    print(train_loss, test_loss)

Epoch: 1/1, Train Loss: 0.7080339193344116, Test Loss 0.7078396118912453, Accuracy: 52.617970265029086
[0.7080339193344116] [0.7078396118912453]


(4188, 2000)   Epoch: 1/1, Train Loss: 0.7196502685546875, Test Loss 0.7007279993811446, Accuracy: 55.08595988538682